In [1]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

In [15]:
df_green = spark.read.parquet('data/pq/green/*/*/')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS nu mber_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [23]:
rdd = df_green \
        .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
        .rdd

In [5]:
from datetime import datetime

In [17]:
start = datetime(year=2020, month=1, day=1)

def filter_date(row):
    return row.lpep_pickup_datetime >= start

In [19]:
rdd \
    .filter(lambda row: row['lpep_pickup_datetime'] >= start).take(1)

# rdd \
#     .filter(filter_date).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)]

In [26]:
row = rdd.take(10)[0]

In [28]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0) #convert to hour

datetime.datetime(2020, 1, 3, 19, 0)

In [33]:
from collections import namedtuple

revenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [34]:
def prepare_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID

    amount = row.total_amount
    count = 1

    key = (hour, zone)
    value = (amount, count)

    return (key, value)

def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

def unwrap_rdd(row):
    return revenueRow(
        hour=row[0][0],
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [35]:
from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [36]:
df_result = rdd \
            .filter(filter_date) \
            .map(prepare_grouping) \
            .reduceByKey(calculate_revenue) \
            .map(unwrap_rdd) \
            .toDF(result_schema)

In [38]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-03 09:00:00| 177|45.989999999999995|    2|
|2020-01-08 10:00:00|  95| 556.0999999999999|   28|
|2020-01-19 10:00:00| 166| 969.2699999999991|   55|
|2020-01-23 17:00:00|  42|376.35000000000014|   28|
|2020-01-14 13:00:00|  82| 379.0600000000001|   20|
|2020-01-03 18:00:00|  82| 753.3899999999996|   54|
|2020-01-27 18:00:00| 193|114.75999999999999|    9|
|2020-01-08 19:00:00| 264|             43.66|    3|
|2020-01-13 11:00:00| 196|             155.8|    7|
|2020-01-02 10:00:00| 260|            294.82|   15|
|2020-01-05 15:00:00|  85| 54.42999999999999|    3|
|2020-01-27 10:00:00| 157|              29.5|    1|
|2020-01-30 13:00:00|  75| 1330.859999999999|   76|
|2020-01-22 05:00:00| 129|              74.0|    6|
|2020-01-22 17:00:00|  82| 716.7999999999996|   44|
|2020-01-06 21:00:00| 215|              31.0|    1|
|2020-01-05 